# Calculate the glass transition temperature, $T_g$, of an atomistic amorphous system using simulation tools at nanoHUB.org

In [1]:
# System setup
import sys, os, tempfile

#sys.path.append('/apps/share64/debian7/ipynb_tools')
sys.path.append('/Storage/davidelbert/persistent/hackathon2017/ben/nanoHUB_remote')

# Run this calculation in a temporary directory
workdir = tempfile.mkdtemp(dir=os.getcwd())
#%cd $workdir

In [2]:
import nanoHUB_remote

# In mysecrets.py, set your web app and account secrets.
# 
# Example, with user credentials:
# auth_data = {
#    'client_id': '',       # Get this when you register a web app
#    'client_secret': '',   # Get this when you register a web app
#    'grant_type': 'password',
#    'username': '',        # Get this when you create a nanoHUB account
#    'password': ''         # Get this when you create a nanoHUB account
# }
#
# This design is strictly for convenience so that this notebook can be 
# shared without storing any secrets.
from nanoHUB_remote.mysecrets import auth_data

# Authenticate; use headers in all subsequent steps
headers = nanoHUB_remote.authenticate(auth_data)

In [3]:
import hublib

ImportError: No module named hublib

In [ ]:
import hubwf

In [ ]:
# Run PolymerModeler to pack chains into a box
import hubwf

# Create an interface to the installed PolymerModeler tool
task = hubwf.RapptureTask('polymod')

# Select the polymer
task.set_input_value('Polymer', 'PMMA (atactic)')

# Set the length of the chains (monomers per chain)
task.set_input_value('Monomers per chain', 10)

# Set the number of chains
task.set_input_value('Number of chains', 30)   

# Have the tool generate input files for LAMMPS
task.set_input_value('Simulation choice', 'LAMMPS input files only')

# Generate LAMMPS commands to relax the structure
task.set_input_value('Minimization levels', 3)
task.set_input_value('Minimization steps', 5000)

# Generate LAMMPS commands to thermalize at 800K
task.set_input_value('Ensemble', 'npt')
task.set_input_value('Number of MD steps', 10000)
task.set_input_value('Temperature', 800, exact=True)

# Run the tool
task.run()
print 'Finished running PolymerModeler'

In [ ]:
# View the initial structure built by PolymerModeler
import nglview as nv
import subprocess as sp

tmp_pdb = 'temp.pdb'
polymod_pdb = 'polymod.pdb'
with open(tmp_pdb, 'w') as f:
    f.write(task.get_output_pdb('Built structures, unwrapped'))
sp.call(['babel', tmp_pdb, polymod_pdb])  # add CONECT records
v = nv.show_structure_file(polymod_pdb)
v.representations = [{'type': 'ball+stick', 'params': {}}]
v.parameters = {'backgroundColor': 'black'}
v

In [ ]:
# Relax and thermalize the initial structure built by PolymerModeler
import hublib.use
%use lammps-15May15

nodes = 2
walltime = '1:00:00'
datafile = 'polymer_relax.data'
with open(datafile, 'w') as f:
    f.write(task.get_output_value('LAMMPS data file'))
infile = 'lammps_relax.in'
with open(infile, 'w') as f:
    f.write(task.get_output_value('LAMMPS input file'))

!submit -n $nodes -w $walltime -i $datafile lammps-15May15-parallel -in $infile

In [ ]:
# Cool and compress the structure with LAMMPS
infile = 'lammps_cool.in'
T_steps = 10000
lammps_input = """
read_restart      restart.lammps

neighbor          2.0 bin
thermo_style      custom temp density vol etotal ke pe ebond eangle edihed eimp evdwl ecoul elong press pxx pyy pzz pxy pxz pyz lx ly lz
thermo            100
thermo_modify     flush yes line multi format float %16.6g

timestep          4
neigh_modify      every 1 delay 5
kspace_style      pppm 1e-4
run_style         respa 3 2 2 bond 1 pair 2 kspace 3
velocity          all create 800.0 1214556 mom yes rot yes dist gaussian

# Loop
variable          i loop 26
label             loopa
reset_timestep    0
variable          T equal 800+($i-1)*(-20)

fix               1 all npt temp $T $T 100.0 iso 1.0 1.0 1000.
dump              2 all custom 500 strcool_$TK.dump id type x y z
run               {0}
write_data        str_cool$TK.data
unfix             1
undump            2
next              i
jump              {1} loopa
""".format(T_steps, infile)

with open(infile, 'w' ) as f:
    f.write(lammps_input)
walltime = '12:00:00'

!submit -n $nodes -w $walltime -i restart.lammps lammps-15May15-parallel -in $infile

In [ ]:
# Extract the density and temperature data from LAMMPS output; find Tg
from Tgcalc import getVDT, getVDTdata, getTg_BL
import matplotlib.pyplot as plt
%matplotlib notebook

getVDT('log.lammps', T_steps)
data_file = 'log.rec_VDT'

# Plot density vs. T
T, rho, err = getVDTdata(data_file)
plt.style.use('ggplot')
plt.plot(T, rho, 'b-')
plt.xlabel('T (K)')
plt.ylabel('Density (g/cc)')
plt.show()

# Calculate Tg
print 'Tg from bilinear fit:', getTg_BL(data_file)